In [16]:
import pandas as pd
import sqlalchemy
import json
import time
import yfinance as yf
from datetime import datetime
import pytz

# Ingest Datalake

In [46]:
# ---- set database connection
database_username = 'gian'
database_password = 'colingewe'
database_ip       = '172.17.0.5'
database_name     = 'gian_database'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                     database_ip, database_name))

In [47]:
lq45 = ["ASII.JK", "BBCA.JK", "BBRI.JK", "BBNI.JK", "BMRI.JK", "UNTR.JK", "TLKM.JK", "INDF.JK", "UNVR.JK", "WIKA.JK", "EXCL.JK", "ADRO.JK", "ANTM.JK", "AALI.JK", "ICBP.JK", "INCO.JK", "ITMG.JK", "JPFA.JK", "JSMR.JK", "KLBF.JK", "LPKR.JK", "MNCN.JK", "PGAS.JK", "PTBA.JK", "PTPP.JK", "SMGR.JK", "SRIL.JK", "SMBR.JK", "TKIM.JK", "TINS.JK"]

In [48]:
dt_now = pd.to_datetime(datetime.now(tz=pytz.timezone('Asia/Jakarta')))

In [49]:
# Set the start and end dates for the data to be downloaded
start_date = "2010-01-01"
end_date = dt_now

all_stocks = []

for stocks in lq45:
    # Download the historical data for BBRI
    temp_data = yf.download(stocks, start=start_date, end=end_date)
    temp_data['Stock Name'] = stocks[:-3]
    all_stocks.append(temp_data)

df_all_stocks = pd.concat(all_stocks).reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [50]:
stocks_df_list = {}
for i in df_all_stocks['Stock Name'].unique():
    temp_df = df_all_stocks[df_all_stocks['Stock Name'] == i].reset_index(drop=True).drop(['Stock Name'],axis=1)
    stocks_df_list[i] = temp_df.sort_values(by="Date",ascending=False)

# Export to Datalake

In [51]:
for i in list(stocks_df_list.keys()):
    stocks_df_list[i].to_sql(con=database_connection, 
          name='dl_stock_{}'.format(i), 
          if_exists='replace',
          index=False)
    print(stocks_df_list[i])

           Date    Open    High     Low   Close    Adj Close     Volume
0    2010-01-04  3530.0  3550.0  3465.0  3530.0  2421.050781         40
1    2010-01-05  3550.0  3570.0  3485.0  3550.0  2434.768066         40
2    2010-01-06  3530.0  3580.0  3515.0  3530.0  2421.050781         40
3    2010-01-07  3420.0  3560.0  3415.0  3420.0  2345.607422         40
4    2010-01-08  3440.0  3450.0  3380.0  3440.0  2359.324463         40
...         ...     ...     ...     ...     ...          ...        ...
3263 2023-02-27  5725.0  5800.0  5700.0  5800.0  5800.000000   28650400
3264 2023-02-28  6200.0  6275.0  6100.0  6100.0  6100.000000  154669700
3265 2023-03-01  6100.0  6175.0  6050.0  6150.0  6150.000000   49851200
3266 2023-03-02  6150.0  6175.0  6075.0  6150.0  6150.000000   26943600
3267 2023-03-03  6150.0  6150.0  6025.0  6050.0  6050.000000   32818500

[3268 rows x 7 columns]
           Date    Open    High     Low   Close    Adj Close     Volume
0    2010-01-04  1010.0  1010.0   965.0

           Date    Open    High     Low   Close    Adj Close    Volume
0    2010-01-04  2210.0  2220.0  2210.0  2210.0  1597.647095   5952500
1    2010-01-05  2280.0  2290.0  2220.0  2280.0  1648.251343        20
2    2010-01-06  2260.0  2290.0  2240.0  2260.0  1633.793335        20
3    2010-01-07  2220.0  2260.0  2210.0  2220.0  1604.876831        20
4    2010-01-08  2230.0  2230.0  2190.0  2230.0  1612.105469        20
...         ...     ...     ...     ...     ...          ...       ...
3264 2023-02-27  4350.0  4350.0  4210.0  4300.0  4300.000000  25283100
3265 2023-02-28  4300.0  4320.0  4180.0  4180.0  4180.000000  56167600
3266 2023-03-01  4170.0  4200.0  4110.0  4150.0  4150.000000  20343000
3267 2023-03-02  4160.0  4220.0  4150.0  4150.0  4150.000000  18391700
3268 2023-03-03  4150.0  4170.0  4050.0  4060.0  4060.000000  25480200

[3269 rows x 7 columns]
           Date        Open        High         Low       Close   Adj Close  \
0    2010-01-04  305.604218  305.604218  300

           Date     Open     High      Low    Close     Adj Close   Volume
0    2010-01-04  32050.0  32250.0  31750.0  32050.0   8621.447266  1226500
1    2010-01-05  33300.0  33500.0  32200.0  33300.0   8957.698242  1552000
2    2010-01-06  33000.0  34300.0  32950.0  33000.0   8876.998047  1745500
3    2010-01-07  32600.0  33500.0  32400.0  32600.0   8769.399414   988500
4    2010-01-08  32650.0  32950.0  32550.0  32650.0   8782.848633  1826000
...         ...      ...      ...      ...      ...           ...      ...
3263 2023-02-27  36025.0  36325.0  35825.0  36050.0  36050.000000  2007000
3264 2023-02-28  36075.0  37725.0  36050.0  37200.0  37200.000000  4923200
3265 2023-03-01  37200.0  37300.0  36750.0  36975.0  36975.000000  2432500
3266 2023-03-02  37175.0  38000.0  36975.0  38000.0  38000.000000  2904700
3267 2023-03-03  38075.0  38500.0  37650.0  37950.0  37950.000000  1869200

[3268 rows x 7 columns]
           Date    Open    High     Low   Close    Adj Close    Volume
0   

           Date    Open    High     Low   Close    Adj Close    Volume
0    2010-01-04  7600.0  7650.0  7500.0  7600.0  5561.376465   3979000
1    2010-01-05  7700.0  7750.0  7650.0  7700.0  5634.552734   5844500
2    2010-01-06  7800.0  7950.0  7700.0  7800.0  5707.729004   4945000
3    2010-01-07  7700.0  7950.0  7600.0  7700.0  5634.552734   3886000
4    2010-01-08  7650.0  7750.0  7600.0  7650.0  5597.965332   9397500
...         ...     ...     ...     ...     ...          ...       ...
3263 2023-02-27  7400.0  7450.0  7250.0  7375.0  7375.000000   9114700
3264 2023-02-28  7400.0  7425.0  7200.0  7225.0  7225.000000  10922900
3265 2023-03-01  7225.0  7325.0  7200.0  7250.0  7250.000000   5717600
3266 2023-03-02  7250.0  7250.0  7150.0  7150.0  7150.000000   7080600
3267 2023-03-03  7150.0  7275.0  7150.0  7175.0  7175.000000   8810300

[3268 rows x 7 columns]
           Date   Open   High    Low  Close   Adj Close    Volume
0    2013-06-17  290.0  290.0  200.0  250.0  227.802200  